# Importer les librairies et les classes nécessaires

In [14]:
import praw
import urllib, urllib.request
import xmltodict
import datetime
import pickle
from os import path
import pandas as pd
from Classes import Document, RedditDocument, ArxivDocument
from Classes import Document
from Classes import Author
from Corpus import Corpus

# Fonction pour afficher la structure hiérarchique d'un dictionnaire

In [15]:
def showDictStruct(d):
    def recursivePrint(d, i):
        for k in d:
            if isinstance(d[k], dict):
                print("-"*i, k)
                recursivePrint(d[k], i+2)
            else:
                print("-"*i, k, ":", d[k])
    recursivePrint(d, 1)

# Configuration de l'API Reddit à l'aide la bibliothèque praw

In [16]:
reddit = praw.Reddit(client_id='wfrwVmRrikEa9yhkXWMzMQ', client_secret='ahqeIx-Xp4pbFgGccGsLrrd1aecWkg', user_agent='projet_python')

## Récupération des 100 publications populaires sur Reddit

In [17]:
limit = 100
hot_posts = reddit.subreddit('all').hot(limit=limit)

### Récupération des données à partir de ces 100 publications populaires

In [18]:
docs = []
docs_bruts = []
afficher_cles = False
for i, post in enumerate(hot_posts):
    if i%10==0: print("Reddit:", i, "/", limit)
    # Pour connaître les différentes variables et leur contenu
    if afficher_cles:  
        for k, v in post.__dict__.items():
            pass
            print(k, ":", v)

    # On ne considère pas les posts sans texte
    if post.selftext != "":  
        pass
    docs.append(post.selftext.replace("\n", " "))
    docs_bruts.append(("Reddit", post))

print (docs)

Reddit: 0 / 100
Reddit: 10 / 100
Reddit: 20 / 100
Reddit: 30 / 100
Reddit: 40 / 100
Reddit: 50 / 100
Reddit: 60 / 100
Reddit: 70 / 100
Reddit: 80 / 100
Reddit: 90 / 100
['', '', '', '', '', '', '', '', '', '', '', '', '', 'Due to a window falling out of an airplane in Portland, my flight today in ecuador was canceled, otherwise I would have missed the civil unrest by a couple hours. ', 'My (M32) sister-in-law (F18) is living with my wife (F27) and I while she attends university. She has a partial scholarship and we let her live here so her family can save money.  My wife was injured severely in a fall while hiking two years ago. Unfortunately she has lost the use of her limbs.  I love my wife very much and I will never divorce her. We have two very young kids that need us both. Also my work provides us with medical insurance which obviously is very important to us.  The crux of the matter comes down to sex. My wife cannot have sex. Not any kind. There are medical reasons. No I won\'t g

# Récupération des résumés de publications ArXiv avec une requête API


In [19]:
# Paramètres
query_terms = ["clustering", "Dirichlet"]
max_results = 50

# Requête
url = f'http://export.arxiv.org/api/query?search_query=all:{"+".join(query_terms)}&start=0&max_results={max_results}'
data = urllib.request.urlopen(url)

# Format dict (OrderedDict)
data = xmltodict.parse(data.read().decode('utf-8'))

#showDictStruct(data)

# Ajout résumés à la liste
# Vérifie si 'entry' existe dans la réponse************
if 'entry' in data['feed']:
    # Si 'entry' existe
    for i, entry in enumerate(data["feed"]["entry"]):
        if i % 10 == 0:
            print("ArXiv:", i, "/", limit)
        docs.append(entry["summary"].replace("\n", " "))
        docs_bruts.append(("ArXiv", entry))
else:
    # Si 'entry' n'existe pas, affichez un message et/ou prenez d'autres mesures
    print(f"Attention : 'entry' n'est pas présent dans les données de réponse pour la requête: {url}")

Attention : 'entry' n'est pas présent dans les données de réponse pour la requête: http://export.arxiv.org/api/query?search_query=all:clustering+Dirichlet&start=0&max_results=50


# Prétraitement et exploitation des données de Reddit et Arxiv

In [20]:
# Affichage et suppression des doublons
print(f"# docs avec doublons : {len(docs)}")
docs = list(set(docs))
print(f"# docs sans doublons : {len(docs)}")

for i, doc in enumerate(docs):
    print(f"Document {i}\t# caractères : {len(doc)}\t# mots : {len(doc.split(' '))}\t# phrases : {len(doc.split('.'))}")
    if len(doc)<100:
        docs.remove(doc)

# Concaténation des documents restants
longueChaineDeCaracteres = " ".join(docs)

# docs avec doublons : 100
# docs sans doublons : 13
Document 0	# caractères : 0	# mots : 1	# phrases : 1
Document 1	# caractères : 95	# mots : 19	# phrases : 1
Document 2	# caractères : 6	# mots : 1	# phrases : 1
Document 3	# caractères : 3268	# mots : 653	# phrases : 59
Document 4	# caractères : 220	# mots : 41	# phrases : 6
Document 5	# caractères : 331	# mots : 37	# phrases : 4
Document 6	# caractères : 470	# mots : 89	# phrases : 7
Document 7	# caractères : 45	# mots : 9	# phrases : 1
Document 8	# caractères : 17	# mots : 4	# phrases : 1


# Factory Pattern pour la création d'instances de documents à partir de données brutes


In [21]:
collection = []
# FACTORY PATTERN
class DocumentFactory:
    @staticmethod
    def create_document(nature, doc):
        if nature == "ArXiv":  # Les fichiers de ArXiv ou de Reddit ne sont pas formatés de la même manière à ce stade.
        #showDictStruct(doc)

            titre = doc["title"].replace('\n', '')  # On enlève les retours à la ligne
            """try:
                authors = ", ".join([a["name"] for a in doc["author"]])  # On fait une liste d'auteurs, séparés par une virgule
            except:
                authors = doc["author"]["name"]  # Si l'auteur est seul, pas besoin de liste"""
            # récupérer l'auteur et les co-auteur
            try:
                authors_list = [a["name"] for a in doc["author"]]
                primary_author = authors_list[0]
                co_authors = authors_list[1:] if len(authors_list) > 1 else []
            except TypeError:
                primary_author = doc["author"]["name"]
                co_authors = []
            summary = doc["summary"].replace("\n", "")  # On enlève les retours à la ligne
            date = datetime.datetime.strptime(doc["published"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y/%m/%d")  # Formatage de la date en année/mois/jour avec librairie datetime

            #return ArxivDocument(titre, authors, date, doc["id"], summary) 
            return ArxivDocument(titre, primary_author, date, doc["id"], summary, co_auteurs=co_authors)# Création du Document
           
        elif nature == "Reddit":
            titre = doc.title.replace("\n", '')
            auteur = str(doc.author)
            date = datetime.datetime.fromtimestamp(doc.created).strftime("%Y/%m/%d")
            url = "https://www.reddit.com/"+doc.permalink
            texte = doc.selftext.replace("\n", "")
            nb_commentaires = doc.num_comments
            return RedditDocument(titre, auteur, date, url, texte, nb_commentaires)
        else:
            raise ValueError("Nature de document non supportée.")

### Utilisation du factory pour créer des objets Document

In [22]:
for nature, doc in docs_bruts:
    doc_classe = DocumentFactory.create_document(nature, doc)
    collection.append(doc_classe)

### Création de l'index de documents

In [23]:
id2doc = {}
for i, doc in enumerate(collection):
    id2doc[i] = doc.titre

### Gestion des Auteurs et création d'une liste d'objets Auteur avec index associé


In [24]:
authors = {}
aut2id = {}
num_auteurs_vus = 0

# Création de la liste et l'index des Auteurs
for doc in collection:
    if doc.auteur not in aut2id:
        num_auteurs_vus += 1
        authors[num_auteurs_vus] = Author(doc.auteur)
        aut2id[doc.auteur] = num_auteurs_vus

    authors[aut2id[doc.auteur]].add(doc.texte)

# Construction du corpus à partir des documents

In [25]:
corpus = Corpus("Mon corpus")

for doc in collection:
    corpus.add(doc)
#corpus.show(tri="abc")
#print(repr(corpus))

# Sauvegarde
# Vérifier si le fichier 'corpus.csv' existe déjà
csv_file_exists = path.exists('corpus.csv')
pickle_file_exists = path.exists('corpus.pkl')

if not pickle_file_exists: 
    import pickle

    # Ouverture d'un fichier, puis écriture avec pickle
    with open("corpus.pkl", "wb") as f:
        pickle.dump(corpus, f)

    # Supression de la variable "corpus"
    del corpus

    # Ouverture du fichier, puis lecture avec pickle
    with open("corpus.pkl", "rb") as f:
        corpus = pickle.load(f)

    # La variable est réapparue
    print("Corpus brut", corpus)

if not csv_file_exists:
    # Création du contenu CSV
    nature=corpus.elements_du_corpus()[0]
    titre=corpus.elements_du_corpus()[1]
    Auteur=corpus.elements_du_corpus()[2]
    Co_Auteurs=corpus.elements_du_corpus()[3]
    nb_commentaires=corpus.elements_du_corpus()[4]
    Date=corpus.elements_du_corpus()[5]
    url=corpus.elements_du_corpus()[6]
    texte=corpus.elements_du_corpus()[7]
    
    # Transformation des données en csv
    df = pd.DataFrame(zip(nature,titre,Auteur,Co_Auteurs,Date,url,texte,nb_commentaires), columns=['Nature','Titre','Auteur','Co_Auteurs','Date','URL','Texte','Nb commentaires'])
    df.to_csv(r'corpus.csv',index=False,sep=';')
    
    print("Creation du CSV OK")

else:
    print("Le corpus et le fichier CSV existent déjà.")

print(corpus)

Corpus brut [OC] Formerly Popular Names that Died Off, Female Edition, par Phanyxx , type: Reddit	Nombre de commentaires : 566
A letter I wrote to myself 10 years ago, par Cantspeakgerman , type: Reddit	Nombre de commentaires : 808
A lot of ants died for my mac n cheese, par fishpilled , type: Reddit	Nombre de commentaires : 559
A recent study concluded that from 1991 to 2016—when most states implemented more restrictive gun laws—gun deaths fell sharply, par _BearHawk , type: Reddit	Nombre de commentaires : 515
Aaaaaaaahh!, par Palleco , type: Reddit	Nombre de commentaires : 113
AITA for kicking out my sister in law for spreading (true) rumors about my marriage., par EnvironmentalGur2853 , type: Reddit	Nombre de commentaires : 1203
Amazon’s Twitch to Cut 500 Employees, About 35% of Staff, par joe4942 , type: Reddit	Nombre de commentaires : 483
Anyone else getting real sick and tired of these stupid big utes?, par Jesh3023 , type: Reddit	Nombre de commentaires : 1108
Anyone else seeing 

# Recherche, concordance et statistique des mots-clés

In [26]:
# Pour rechercher le mot-clé "..."
results = corpus.search('year')
for result in results:
    print("recherche:", result)

# Méthode concorde pour rechercher "motif recherche"
expression_recherchee = "from"
context_size = 20  # Nombre de caractères avant et après le terme pour le contexte
concordance_results = corpus.concorde(expression_recherchee, context_size)
print("concordance:", concordance_results)

# Statistiques textuelles
freq_table = corpus.stats(10)

Recherche du terme: year
Cache du texte construit
Nombre de correspondances trouvées: 2
recherche: year
recherche: year
concordance:        contexte gauche motif trouvé       contexte droit
0  s having to commute         from  her parents home al
1  itely took the idea         from   this creators swea
Nombre de mots différents dans le corpus : 491
Les 10 mots les plus fréquents sont:
     mot  term_frequency  doc_frequency
18     i              51              8
1     to              41              7
22   the              32              8
37   and              31              5
11    my              29              5
39   she              27              2
9     in              21              4
2      a              20              4
47   her              17              1
74  that              17              5


# Moteur de recherche

In [27]:
# Construction du vocabulaire
vocab = corpus.build_vocab()

# Test : Affichage des premiers éléments pour vérifier
for word, info in list(vocab.items())[:10]:  # Modifier ce nombre pour voir plus ou moins de résultats
    print(f"Mot: {word}, ID: {info['id']}, Occurrences totales: {info['total_occurrences']}")

# Vous pouvez également tester pour un mot spécifique
test_word = 'exemple'  # Remplacez ceci par un mot de votre choix qui est susceptible d'être dans votre corpus
if test_word in vocab:
    print(f"Test pour le mot '{test_word}':", vocab[test_word])
else:
    print(f"Le mot '{test_word}' n'est pas dans le vocabulaire.")

# Construction de la matrice TF
tf_matrix = corpus.build_tf_matrix()
#print(tf_matrix)

# Mise à jour du vocabulaire avec le nombre total d’occurrences et le document count
mise = corpus.update_vocab_with_doc_frequency()

# Construction de la matrice TFxIDF
mat_TFxIDF = corpus.build_tfidf_matrix()
print(mat_TFxIDF)

Mot: 1210368808, ID: 376, Occurrences totales: 1
Mot: 3, ID: 239, Occurrences totales: 2
Mot: 4, ID: 150, Occurrences totales: 1
Mot: 5, ID: 122, Occurrences totales: 1
Mot: a, ID: 10, Occurrences totales: 20
Mot: a36pvco10rvhadji, ID: 366, Occurrences totales: 1
Mot: about, ID: 280, Occurrences totales: 6
Mot: accident, ID: 269, Occurrences totales: 1
Mot: accord, ID: 457, Occurrences totales: 1
Mot: added, ID: 448, Occurrences totales: 1
Le mot 'exemple' n'est pas dans le vocabulaire.
Le mot 'portland,' n'est pas dans le vocabulaire.
Le mot 'canceled,' n'est pas dans le vocabulaire.
Le mot 'hours.' n'est pas dans le vocabulaire.
Le mot '(m32)' n'est pas dans le vocabulaire.
Le mot 'sister-in-law' n'est pas dans le vocabulaire.
Le mot '(f18)' n'est pas dans le vocabulaire.
Le mot '(f27)' n'est pas dans le vocabulaire.
Le mot 'university.' n'est pas dans le vocabulaire.
Le mot 'money.my' n'est pas dans le vocabulaire.
Le mot 'ago.' n'est pas dans le vocabulaire.
Le mot 'limbs.i' n'est 